This is your private sketch-notebook. Do whatever you want with it. When code is ready to be developed, please just copy and paste it to the final notebook. 

In [2]:
import sys
kaggle_path = '../../../Data/Kaggle/train/*'
sys.path.append('../../..')
from First_start.imports import *
from First_start.multi_gpu import *
from First_start.init_vars import *
from First_start.start_functions import *
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [3]:
# Get x and y variables
x_train, y_train, x_test, y_test, image_df, dummy_df = get_xy(folders_path=kaggle_path, test_size=0.3,
                                                              img_size=(224, 224), seed=seed)

# Let's check the shape of those variables
print (x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# Own Code

In [130]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, 224, 224), border_mode='same', activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
# Compile model
epochs = 1 # dont leave it like this, only for testing!
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_9 (Convolution2D)  (None, 32, 224, 224)  896         convolution2d_input_5[0][0]      
____________________________________________________________________________________________________
dropout_9 (Dropout)              (None, 32, 224, 224)  0           convolution2d_9[0][0]            
____________________________________________________________________________________________________
convolution2d_10 (Convolution2D) (None, 32, 224, 224)  9248        dropout_9[0][0]                  
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 32, 112, 112)  0           convolution2d_10[0][0]           
___________________________________________________________________________________________

In [131]:
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
classes = model.predict_proba(x_test, batch_size=32)

# get probability table
proba_df = pd.DataFrame(classes)
proba_df.columns = image_df.groupby('fish_type').count().index # workaround to get column names quickly and in right order

print("Accuracy: %.2f%%" % (scores[1]*100))
print ' '
proba_df

Train on 90 samples, validate on 11 samples
Epoch 1/1
11/11 [==============================] - 0s
Accuracy: 54.55%
 


fish_type,ALB,ARK,BET,DOL,HER,LAG,NoF,YFT
0,0.506756,0.024877,0.058784,0.065291,0.051306,0.038819,0.109125,0.145042
1,0.428844,0.023427,0.058552,0.071125,0.041804,0.033080,0.154019,0.189149
2,0.437535,0.022948,0.063415,0.064843,0.045019,0.035351,0.128865,0.202024
3,0.609962,0.009597,0.037177,0.039952,0.025592,0.016731,0.098356,0.162632
4,0.488781,0.027245,0.060165,0.067221,0.048789,0.037209,0.107406,0.163185
5,0.448581,0.025906,0.067605,0.069368,0.052836,0.039578,0.117511,0.178615
6,0.397859,0.036983,0.075243,0.079164,0.065603,0.052575,0.127660,0.164914
7,0.408537,0.033430,0.072096,0.076143,0.060677,0.048232,0.127109,0.173777
8,0.389521,0.016347,0.062469,0.062859,0.036389,0.025675,0.118288,0.288453
9,0.344012,0.035175,0.081605,0.091208,0.060461,0.049630,0.136820,0.201089


In [142]:
proba_df['img_names'] = labels_test.values

fish_type,ALB,ARK,BET,DOL,HER,LAG,NoF,YFT,img_names
0,0.506756,0.024877,0.058784,0.065291,0.051306,0.038819,0.109125,0.145042,img_05281.jpg
1,0.428844,0.023427,0.058552,0.071125,0.041804,0.033080,0.154019,0.189149,img_01118.jpg
2,0.437535,0.022948,0.063415,0.064843,0.045019,0.035351,0.128865,0.202024,img_07869.jpg
3,0.609962,0.009597,0.037177,0.039952,0.025592,0.016731,0.098356,0.162632,img_01287.jpg
4,0.488781,0.027245,0.060165,0.067221,0.048789,0.037209,0.107406,0.163185,img_01144.jpg
5,0.448581,0.025906,0.067605,0.069368,0.052836,0.039578,0.117511,0.178615,img_02519.jpg
6,0.397859,0.036983,0.075243,0.079164,0.065603,0.052575,0.127660,0.164914,img_05043.jpg
7,0.408537,0.033430,0.072096,0.076143,0.060677,0.048232,0.127109,0.173777,img_04851.jpg
8,0.389521,0.016347,0.062469,0.062859,0.036389,0.025675,0.118288,0.288453,img_05282.jpg
9,0.344012,0.035175,0.081605,0.091208,0.060461,0.049630,0.136820,0.201089,img_01156.jpg
